# IMDB Movie Review Dataset

This is a library to download and parse the [IMDB's Large Movie Review Dataset](http://ai.stanford.edu/~amaas/data/sentiment/) dataset and a demo of a transformer based model. The dataset has 25K training, and 25K test dataset, plus 50K unlabeled examples.

It's inspired on [Keras' Text classification with Transformer](https://keras.io/examples/nlp/text_classification_with_transformer/) demo.




## Environment Set Up

Let's set up `go.mod` to use the local copy of GoMLX, so it can be developed jointly the dataset code with the model. That's often how data pre-processing and model code is developed together with experimentation.

If you are not changing code, feel free to simply skip this cell. Or if you used a different directory for you projects, change it below.

Notice the directory `${HOME}/Projects/gomlx` is where the GoMLX code is copied by default in [its Docker](https://hub.docker.com/repository/docker/janpfeifer/gomlx_jupyterlab/general).

In [1]:
!*go mod edit -replace github.com/gomlx/gomlx="${HOME}/Projects/gomlx"

## Data Preparation

### Downloading data files

To download, uncompress and untar to the local directory, simply do the following. Notice if it's already downloaded in the given `--data` directory, it returns immediately.

In [2]:
import (
    "github.com/gomlx/gomlx/examples/imdb"
    "github.com/gomlx/gomlx/ml/data"
)

var flagDataDir = flag.String("data", "~/work/imdb", "Directory to cache downloaded and generated dataset files.")

func AssertNoError(err error) {
    if err != nil {
        log.Fatalf("Failed: %+v", err)
    }
}

func AssertDownloaded() {
    *flagDataDir = data.ReplaceTildeInDir(*flagDataDir)
    if !data.FileExists(*flagDataDir) {
        AssertNoError(os.MkdirAll(*flagDataDir, 0777))
    }

    AssertNoError(imdb.Download(*flagDataDir))
}

%%
AssertDownloaded()

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.


### Sampling some examples

It creates a small dataset and print out some random examples.

It also defines the `DType`, used for all internal representations of the model, and the flag `--max_len` that defines the maximum number of tokens used per observation. This will beused in the modeling later.

In [3]:
import (
    "github.com/gomlx/gomlx/examples/imdb"
)

// DType used for the models.
const DType = shapes.Float32

var (
    flagMaxLen              = flag.Int("max_len", 200, "Maximum number of tokens to take from observation.")
)

func Sample() {
    ds := imdb.NewDataset("Test", imdb.Test, *flagMaxLen, 3, DType, true, nil)
    _, inputs, labels, err := ds.Yield()
    AssertNoError(err)
    labelsData := shapes.CastAsDType(labels[0].Local().Data(), shapes.Int64).([]int)
    for ii := 0; ii < 3; ii++ {
        fmt.Printf("\nlabel=%v, input=%q\n", labelsData[ii], imdb.InputToString(inputs[0], ii))
    }
    fmt.Println()
}

%% --max_len=200
AssertDownloaded()
Sample()


> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.

label=0, input="a babysitting job and during the reading of jack the beanstalk as a bedtime story with the kid reading it to costello costello s jack falls asleep and dreams himself into the story there s a wizard of oz kind of feel to the story in that the characters in the dream are all the equivalents of real life acquaintances of jack and the movie opens in black white and shifts to colour during the dream sequence the fight scenes between jack and the giant and the dance scene between jack and polly dorothy ford are among the amusing parts of the movie polly of course also leads to one of the questions of the movie what happened to her jack and gang apparently left her behind in the giant s castle i know it was just a dream so who cares still i wondered there were also a couple of cute song and dance routines my 4 year old giggl

## Training

We will create 3 different types of models for this demo: **Bag of Words** (or simply **bow**), **CNNs** and **Transformers**.

### Model Support

We first define here a few componens shared among all models:

* `Normalize` according to flag, works for sequence nodes (shaped `[batch_size, sequence_len, embedding_dim]`) and feature nodes (shaped `[batch_size, embedding_dim]`).
* `EmbedTokensGraph` that embeds the tokens, before being consumed by the models.
* `ReadoutGraph` that takes the embeddings after they are pooled on the sequence axis (so shaped `[batch_size, embedding_dim]` it adds a FNN (feed-forward neural network) with a few layers and convert to the final logits.

We also define the corresponding hyperparameters.

In [4]:
import (
    . "github.com/gomlx/gomlx/graph"
    
    "github.com/gomlx/gomlx/ml/context"
    "github.com/gomlx/gomlx/types/shapes"
)

var (
    flagNormalization    = flag.String("norm", "layer", "Type of normalization to use. Valid values are \"none\", \"batch\", \"layer\".")
    flagDropoutRate      = flag.Float64("dropout", 0.15, "Dropout rate")
    flagMaxVocab            = flag.Int("max_vocab", 20000, "Top most frequent words to consider, the rest is considered unknown.")
    flagTokenEmbeddingSize  = flag.Int("token_embed", 32, "Size of token embedding table. There are ~140K unique tokens")
    flagNumHiddenLayers     = flag.Int("hidden_layers", 2, "Number of output hidden layers, stacked with residual connection.")
    flagNumNodes            = flag.Int("num_nodes", 32, "Number of nodes in output hidden layers.")
    flagWordDropoutRate  = flag.Float64("word_dropout", 0, "Dropout rate for whole words of the input")
)

// Normalize x according to --norm flag. Works for sequence nodes (rank-3) or plain feature nodes (rank-2).
func Normalize(ctx *context.Context, x *Node) *Node {
    switch *flagNormalization {
    case "layer":
        if x.Rank() == 3 {
            // Normalize sequence.
            return layers.LayerNormalization(ctx, x, -2, -1).
                LearnedOffset(true).LearnedScale(true).ScaleNormalization(true).Done()
        } else {
            // Normalize features only.
            return layers.LayerNormalization(ctx, x, -1).Done()
        }
    case "batch":
        return layers.BatchNormalization(ctx, x, -1).Done()
    case "none":
        return x
    }
    g := x.Graph()
    g.SetErrorf("invalid normalization selected %q -- valid values are batch, layer, none", *flagNormalization)
    return g.InvalidNode()
}

// EmbedTokensGraph creates embeddings for tokens and returns them along with the mask of used tokens --
// set to false where padding was used.
func EmbedTokensGraph(ctx *context.Context, tokens *Node) (embed, mask *Node) {
    g := tokens.Graph()
    mask = NotEqual(tokens, ZerosLike(tokens)) // Mask of tokens actually used.

    // The token ids are indexed by frequency. Truncate to the vocabulary size considered, replacing
    // ids higher than that by 0.
    maxVocab := len(imdb.LoadedVocab.ListEntries)
    if maxVocab > *flagMaxVocab {
        maxVocab = *flagMaxVocab
    }

    // Limits tokens to the maxVocab.
    tokens = Where(GreaterOrEqual(tokens, Const(g, maxVocab)),
        MulScalar(OnesLike(tokens), float64(maxVocab-1)),
        tokens)

    // Embed tokens: shape=[batchSize, maxLen, embedDim]
    embed = layers.Embedding(ctx.In("tokens"), tokens, DType, maxVocab, *flagTokenEmbeddingSize)
    embed = Where(mask, embed, ZerosLike(embed))
    
    if *flagWordDropoutRate > 0 {
        dims := embed.Shape().Dimensions[:len(embed.Shape().Dimensions)-1]
        dropoutMask := Ones(g, shapes.Make(DType, dims...))
        dropoutMask = layers.Dropout(ctx, dropoutMask, ConstAsDType(g, DType, *flagWordDropoutRate))
        dropoutMask = ExpandDims(dropoutMask, -1)
        embed = Mul(embed, dropoutMask)
    }    
    return
}

// Activation function used for models.
func Activation(x *Node) *Node {
    return layers.Relu(x)
}

// ReadoutGraph takes the embeddings after they have been pooled on the sequence axis, so shaped `[batch_size, embed_dim]`
// adds a FNN on top and readout the final logit.
func ReadoutGraph(ctx *context.Context, embed *Node) *Node {
    g := embed.Graph()
    if !g.Ok() {
        return g.InvalidNode()
    }
    var dropoutRate *Node
    if *flagDropoutRate > 0 {
        dropoutRate = ConstAsDType(g, DType, *flagDropoutRate)
    }

    // Output layers.
    for ii := 0; ii < *flagNumHiddenLayers; ii++ {
        ctx := ctx.In(fmt.Sprintf("output_dense_%d", ii))
        residual := embed
        if *flagDropoutRate > 0 {
            embed = layers.Dropout(ctx, embed, dropoutRate)
        }
        //embed = Tanh(embed)
        embed = Activation(embed)
        embed = layers.DenseWithBias(ctx, embed, *flagNumNodes)
        embed = Normalize(ctx, embed)
        if ii > 0 {
            // Add residual connection.
            embed = Add(embed, residual)
        }
    }

    // Final embed layer with dimension 1.
    {
        ctx := ctx.In("readout")
        if *flagDropoutRate > 0 {
            embed = layers.Dropout(ctx, embed, dropoutRate)
        }
        embed = Activation(embed)
        embed = layers.DenseWithBias(ctx, embed, 1)
    }
    return embed
}

### Bag Of Words Model

This is the simples model we are going to train. It's basically a no-op that connects the embedding table we defined above with the `ReadoutGraph` FNN afterwards.

We define here placeholders for our future *CNN* and *Transformer* models.

Finally we test that the shape is correct. To actually train and evaluate we still need to define the training loop, what we do in the following section.

In [5]:
import (
    . "github.com/gomlx/gomlx/graph"
    
    "github.com/gomlx/gomlx/ml/context"
)

var (
    // ML Manager creation flags.
    flagNumThreads  = flag.Int("num_threads", -1, "Number of threads. Leave as -1 to use as many as there are cores.")
    flagNumReplicas = flag.Int("num_replicas", 1, "Number of replicas.")
    flagPlatform    = flag.String("platform", "", "Platform to use, if empty uses the default one.")

    // Model type.
    flagModel            = flag.String("model", "transformer", "Model type: bow or transformer.")
)

// ModelGraph builds the model for our demo. It returns the logits, not the predictions, which works with most losses.
func ModelGraph(ctx *context.Context, spec any, inputs []*Node) []*Node {
    _ = spec // Not used.
    tokens := inputs[0]
    g := tokens.Graph()
    if !g.Ok() {
        return nil
    }
    embed, mask := EmbedTokensGraph(ctx, tokens)

    // Normalization function.
    if *flagModel == "bow" {
        // Bag-Of-Words model doesn't do anything, it's just the embedding table for each token.
    } else if *flagModel == "cnn" {
        embed = Conv1DGraph(ctx, embed, mask)
    } else if *flagModel == "transformer" {
        embed = TransformerGraph(ctx, tokens, embed, mask)
    } else {
        g.SetErrorf("unknown model type %q, only types \"bow\", \"cnn\" and \"transformer\" are implemented", *flagModel)
        return nil
    }

    // Sum-up per-token embeddings and do a FNN on the output. From now on, the dimensions are `[batch_dim, embed_dim]`
    // Notice we are not using mask.
    embed = ReduceMax(embed, 1)
    logits := ReadoutGraph(ctx, embed)
    return []*Node{logits}
}

func Conv1DGraph(ctx *context.Context, embed, mask *Node) *Node {
    log.Panic("Not implemented.")
    return nil
}

func TransformerGraph(ctx *context.Context, tokens, embed, mask *Node) *Node {
    log.Panic("Not implemented.")
    return nil
}

%% --model=bow --platform=Host

// Let's test that the logits are coming out with the right shape: we want [batch_size, 10], since there are 10 classes.
AssertDownloaded()
manager := BuildManager().NumThreads(*flagNumThreads).NumReplicas(*flagNumReplicas).Platform(*flagPlatform).MustDone()
ds := imdb.NewDataset("Test", imdb.Test, *flagMaxLen, 3, DType, true, nil)
_, inputs, _, err := ds.Yield()
AssertNoError(err)

g := manager.NewGraph("test")
ctx := context.NewContext(manager)
logits := ModelGraph(ctx, nil, []*Node{Const(g, inputs[0])})
AssertNoError(g.Error())
AssertNoError(ctx.Error())
fmt.Printf("Logits shape for batch_size=%d: %s\n", 3, logits[0].Shape())

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Logits shape for batch_size=3: (Float32)[3 1]


### Training Loop

Training loop, with the usual bells and whistles. We also do a quick test with 100 steps to check things are working.

In [6]:
import (
    . "github.com/gomlx/gomlx/graph"
    
    "github.com/gomlx/gomlx/ml/context"
    "github.com/gomlx/gomlx/ml/train"
)

var (
    flagOptimizer        = flag.String("optimizer", "adam", "Optimizer, options: adam or sgd.")
    flagNumSteps         = flag.Int("steps", 5000, "Number of gradient descent steps to perform")
    flagBatchSize        = flag.Int("batch", 32, "Batch size for training")
    flagLearningRate     = flag.Float64("learning_rate", 0.0001, "Initial learning rate.")
    flagL2Regularization = flag.Float64("l2_reg", 0, "L2 regularization on kernels. It doesn't interact well with --batch_norm or with --optimizer=adam.")
    
    flagCheckpoint       = flag.String("checkpoint", "", "Directory save and load checkpoints from. If left empty, no checkpoints are created.")
    flagCheckpointKeep   = flag.Int("checkpoint_keep", 10, "Number of checkpoints to keep, if --checkpoint is set.")

    // Training UI:
    flagUseProgressBar = flag.Bool("bar", true, "If to display a progress bar during training")
    flagNumPlotPoints  = flag.Int("plot_points", 50, "Number points to plot.")
)

func trainModel() {
    // Make sure data is downloaded.
    AssertDownloaded()

    // Manager handles creation of ML computation graphs, accelerator resources, etc.
    manager := BuildManager().NumThreads(*flagNumThreads).NumReplicas(*flagNumReplicas).Platform(*flagPlatform).MustDone()
    fmt.Printf("Platform: %s\n", manager.Platform())

    // Datasets.
    var trainDS, trainEvalDS, testEvalDS train.Dataset
    trainDS = imdb.NewDataset("train", imdb.Train, *flagMaxLen, *flagBatchSize, DType, true, nil)
    trainEvalDS = imdb.NewDataset("train-eval", imdb.Train, *flagMaxLen, *flagBatchSize, DType, false, nil)
    testEvalDS = imdb.NewDataset("test-eval", imdb.Test, *flagMaxLen, *flagBatchSize, DType, false, nil)

    // Parallelize generation of batches, to prevent the dataset from being a bottleneck to the acceleartor (GPU/TPU).
    trainDS = data.Parallel(trainDS)
    trainEvalDS = data.Parallel(trainEvalDS)
    testEvalDS = data.Parallel(testEvalDS)

    // Metrics we are interested.
    meanAccuracyMetric := metrics.NewMeanBinaryLogitsAccuracy("Mean Accuracy", "#acc")
    movingAccuracyMetric := metrics.NewMovingAverageBinaryLogitsAccuracy("Moving Average Accuracy", "~acc", 0.01)

    // Context holds the variables and hyperparameters for the model.
    ctx := context.NewContext(manager)
    ctx.SetParam(optimizers.LearningRateKey, *flagLearningRate)
    ctx.SetParam(layers.L2RegularizationKey, *flagL2Regularization)

    // Checkpoints saving.
    var checkpoint *checkpoints.Handler
    if *flagCheckpoint != "" {
        checkpointPath := data.ReplaceTildeInDir(*flagCheckpoint)
        if !path.IsAbs(checkpointPath) {
            checkpointPath = path.Join(*flagDataDir, checkpointPath)
        }
        var err error
        checkpoint, err = checkpoints.Build(ctx).Dir(checkpointPath).Keep(*flagCheckpointKeep).Done()
        AssertNoError(err)
    }

    // Create a train.Trainer: this object will orchestrate running the model, feeding
    // results to the optimizer, evaluating the metrics, etc. (all happens in trainer.TrainStep)
    loss := losses.BinaryCrossentropyLogits
    trainer := train.NewTrainer(
        manager, ctx, ModelGraph, loss,
        optimizers.MustOptimizerByName(*flagOptimizer),
        []metrics.Interface{movingAccuracyMetric}, // trainMetrics
        []metrics.Interface{meanAccuracyMetric})   // evalMetrics

    // Use standard training loop.
    loop := train.NewLoop(trainer)
    if *flagUseProgressBar {
        commandline.AttachProgressBar(loop) // Attaches a progress bar to the loop.
    }

    if checkpoint != nil {
        train.NTimesDuringLoop(loop, 2*(*flagCheckpointKeep), "checkpointing", 100, func(_ *train.Loop, _ []tensor.Tensor) error {
            return checkpoint.Save()
        })
    }

    // Attach a margaid plots.
    if *flagNumPlotPoints > 0 {
        margaid.New(1024, 400, trainEvalDS, testEvalDS).DynamicUpdates().Attach(loop, *flagNumPlotPoints)
    }

    // Loop for given number of steps.
    _, err := loop.RunSteps(trainDS, *flagNumSteps)
    AssertNoError(err)

    // Report final evaluation.
    fmt.Println()
    err = commandline.ReportEval(trainer, trainEvalDS, testEvalDS)
    AssertNoError(err)
}

%% --platform=Host --model=bow --steps=100 --plot_points=0
trainModel()


> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Platform: Host
Training (100 steps):  100% [========================================] (156 steps/s) [loss=0.671] [~loss=0.685] [~acc=56.34%]        

Results on train-eval [Parallelized]:
	Mean Loss (#loss): 0.700
	Mean Accuracy (#acc): 50.00%
Results on test-eval [Parallelized]:
	Mean Loss (#loss): 0.700
	Mean Accuracy (#acc): 50.00%


### Training Bag-Of-Words ("bow") Model

A proper run of our *bow* model.

In [7]:
%% --model=bow --steps=20000 --optimizer=adamw
trainModel()

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Platform: CUDA


Training (20000 steps):  100% [========================================] (746 steps/s) [loss=0.373] [~loss=0.233] [~acc=91.08%]        


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,5.200000e+00 L3.461121e+01,5.222336e+00 L5.158486e+01,5.617841e+01 L6.855851e+01,1.599663e+02 L8.553216e+01,1.588051e+02 L1.025058e+02,1.689651e+02 L1.194795e+02,1.646555e+02 L1.364531e+02,1.764008e+02 L1.534268e+02,1.780756e+02 L1.704004e+02,1.770931e+02 L1.873741e+02,1.788571e+02 L2.043477e+02,1.856230e+02 L2.213214e+02,1.880122e+02 L2.382950e+02,1.834570e+02 L2.552686e+02,1.916073e+02 L2.722423e+02,1.860026e+02 L2.892159e+02,1.906471e+02 L3.061896e+02,1.883918e+02 L3.231632e+02,1.876326e+02 L3.401369e+02,1.968101e+02 L3.571105e+02,1.976586e+02 L3.740842e+02,1.932374e+02 L3.910578e+02,1.968771e+02 L4.080315e+02,1.989091e+02 L4.250051e+02,1.920539e+02 L4.419788e+02,1.941528e+02 L4.589524e+02,1.961179e+02 L4.759261e+02,1.974353e+02 L4.928997e+02,1.961179e+02 L5.098734e+02,1.961625e+02 L5.268470e+02,1.962965e+02 L5.438207e+02,1.914956e+02 L5.607943e+02,1.940412e+02 L5.777680e+02,1.989761e+02 L5.947416e+02,1.991324e+02 L6.117153e+02,1.946664e+02 L6.286889e+02,1.966984e+02 L6.456626e+02,1.993333e+02 L6.626362e+02,1.994003e+02 L6.796099e+02,1.949567e+02 L6.965835e+02,2.000032e+02 L7.135572e+02,1.940635e+02 L7.305308e+02,1.956266e+02 L7.475045e+02,1.984625e+02 L7.644781e+02,1.983732e+02 L7.814518e+02,1.989984e+02 L7.984254e+02,1.999809e+02 L8.153991e+02,1.963188e+02 L8.323727e+02,1.989761e+02 L8.493464e+02,1.980159e+02 L8.663200e+02,1.981052e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,5.200000e+00 L3.461121e+01,5.289328e+00 L5.158486e+01,5.861233e+01 L6.855851e+01,1.699476e+02 L8.553216e+01,1.663526e+02 L1.025058e+02,1.783882e+02 L1.194795e+02,1.756193e+02 L1.364531e+02,1.869404e+02 L1.534268e+02,1.904908e+02 L1.704004e+02,1.916073e+02 L1.873741e+02,1.938402e+02 L2.043477e+02,2.040225e+02 L2.213214e+02,2.074836e+02 L2.382950e+02,2.045584e+02 L2.552686e+02,2.132447e+02 L2.722423e+02,2.090244e+02 L2.892159e+02,2.148300e+02 L3.061896e+02,2.137136e+02 L3.231632e+02,2.140932e+02 L3.401369e+02,2.238959e+02 L3.571105e+02,2.259278e+02 L3.740842e+02,2.234716e+02 L3.910578e+02,2.264414e+02 L4.080315e+02,2.309073e+02 L4.250051e+02,2.253026e+02 L4.419788e+02,2.286967e+02 L4.589524e+02,2.317335e+02 L4.759261e+02,2.334976e+02 L4.928997e+02,2.323364e+02 L5.098734e+02,2.330287e+02 L5.268470e+02,2.347927e+02 L5.438207e+02,2.297239e+02 L5.607943e+02,2.347034e+02 L5.777680e+02,2.405760e+02 L5.947416e+02,2.424294e+02 L6.117153e+02,2.383208e+02 L6.286889e+02,2.402188e+02 L6.456626e+02,2.470740e+02 L6.626362e+02,2.456895e+02 L6.796099e+02,2.402634e+02 L6.965835e+02,2.486370e+02 L7.135572e+02,2.422731e+02 L7.305308e+02,2.452206e+02 L7.475045e+02,2.500661e+02 L7.644781e+02,2.489050e+02 L7.814518e+02,2.511379e+02 L7.984254e+02,2.548000e+02 L8.153991e+02,2.508253e+02 L8.323727e+02,2.540185e+02 L8.493464e+02,2.525000e+02 L8.663200e+02,2.537058e+02 "/> <path d="M1.768000e+01,5.756285e+01 L3.461121e+01,1.542746e+01 L5.158486e+01,2.043358e+01 L6.855851e+01,5.395128e+01 L8.553216e+01,1.088780e+02 L1.025058e+02,1.265000e+02 L1.194795e+02,1.455137e+02 L1.364531e+02,1.483933e+02 L1.534268e+02,1.532828e+02 L1.704004e+02,1.570681e+02 L1.873741e+02,1.666989e+02 L2.043477e+02,1.654691e+02 L2.213214e+02,1.686452e+02 L2.382950e+02,1.724526e+02 L2.552686e+02,1.831011e+02 L2.722423e+02,1.794205e+02 L2.892159e+02,1.855713e+02 L3.061896e+02,1.844825e+02 L3.231632e+02,1.884681e+02 L3.401369e+02,1.908511e+02 L3.571105e+02,1.944630e+02 L3.740842e+02,1.966993e+02 L3.910578e+02,1.981850e+02 L4.080315e+02,1.954416e+02 L4.250051e+02,1.995468e+02 L4.419788e+02,2.038600e+02 L4.589524e+02,2.065744e+02 L4.759261e+02,2.060306e+02 L4.928997e+02,2.052350e+02 L5.098734e+02,2.052668e+02 L5.268470e+02,2.102871e+02 L5.438207e+02,2.099136e+02 L5.607943e+02,2.125930e+02 L5.777680e+02,2.157002e+02 L5.947416e+02,2.177420e+02 L6.117153e+02,2.164611e+02 L6.286889e+02,2.162730e+02 L6.456626e+02,2.184580e+02 L6.626362e+02,2.214571e+02 L6.796099e+02,2.220977e+02 L6.965835e+02,2.214837e+02 L7.135572e+02,2.211038e+02 L7.3053


Results on train-eval [Parallelized]:


<path d="M1.768000e+01,2.548000e+02 L3.461121e+01,2.539523e+02 L5.158486e+01,2.514733e+02 L6.855851e+01,2.366617e+02 L8.553216e+01,1.682559e+02 L1.025058e+02,1.496578e+02 L1.194795e+02,1.484039e+02 L1.364531e+02,1.363072e+02 L1.534268e+02,1.317326e+02 L1.704004e+02,1.317533e+02 L1.873741e+02,1.280395e+02 L2.043477e+02,1.192378e+02 L2.213214e+02,1.176621e+02 L2.382950e+02,1.206397e+02 L2.552686e+02,1.111124e+02 L2.722423e+02,1.171848e+02 L2.892159e+02,1.106248e+02 L3.061896e+02,1.134435e+02 L3.231632e+02,1.144904e+02 L3.401369e+02,1.033035e+02 L3.571105e+02,1.031584e+02 L3.740842e+02,1.071697e+02 L3.910578e+02,1.031376e+02 L4.080315e+02,1.004165e+02 L4.250051e+02,1.092497e+02 L4.419788e+02,1.060497e+02 L4.589524e+02,1.028269e+02 L4.759261e+02,1.004745e+02 L4.928997e+02,1.027169e+02 L5.098734e+02,1.043333e+02 L5.268470e+02,1.039060e+02 L5.438207e+02,1.113925e+02 L5.607943e+02,1.084549e+02 L5.777680e+02,1.010275e+02 L5.947416e+02,1.014267e+02 L6.117153e+02,1.073203e+02 L6.286889e+02,1.053251e+02 L6.456626e+02,1.009960e+02 L6.626362e+02,1.018561e+02 L6.796099e+02,1.071621e+02 L6.965835e+02,1.017170e+02 L7.135572e+02,1.085032e+02 L7.305308e+02,1.087145e+02 L7.475045e+02,1.054784e+02 L7.644781e+02,1.063610e+02 L7.814518e+02,1.050951e+02 L7.984254e+02,1.030438e+02 L8.153991e+02,1.090830e+02 L8.323727e+02,1.038399e+02 L8.493464e+02,1.075805e+02 L8.663200e+02,1.095527e+02 " vector-effect="non-scaling-stroke"/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.547875e+02 L3.461121e+01,2.538408e+02 L5.158486e+01,2.512180e+02 L6.855851e+01,2.353676e+02 L8.553216e+01,1.619847e+02 L1.025058e+02,1.401628e+02 L1.194795e+02,1.365226e+02 L1.364531e+02,1.219833e+02 L1.534268e+02,1.153574e+02 L1.704004e+02,1.133216e+02 L1.873741e+02,1.076296e+02 L2.043477e+02,9.577565e+01 L2.213214e+02,9.193062e+01 L2.382950e+02,9.329784e+01 L2.552686e+02,8.005187e+01 L2.722423e+02,8.568585e+01 L2.892159e+02,7.668089e+01 L3.061896e+02,7.650301e+01 L3.231632e+02,7.731944e+01 L3.401369e+02,6.279031e+01 L3.571105e+02,5.912067e+01 L3.740842e+02,6.122273e+01 L3.910578e+02,5.593792e+01 L4.080315e+02,5.264182e+01 L4.250051e+02,5.750962e+01 L4.419788e+02,5.336155e+01 L4.589524e+02,4.786382e+01 L4.759261e+02,4.435303e+01 L4.928997e+02,4.650060e+01 L5.098734e+02,4.446289e+01 L5.268470e+02,4.192915e+01 L5.438207e+02,4.694987e+01 L5.607943e+02,4.038000e+01 L5.777680e+02,3.151718e+01 L5.947416e+02,2.903503e+01 L6.117153e+02,3.390418e+01 L6.286889e+02,3.080470e+01 L6.456626e+02,2.234082e+01 L6.626362e+02,2.291706e+01 L6.796099e+02,2.942135e+01 L6.965835e+02,1.737438e+01 L7.135572e+02,2.483699e+01 L7.305308e+02,2.115445e+01 L7.475045e+02,1.408821e+01 L7.644781e+02,1.454903e+01 L7.814518e+02,1.101033e+01 L7.984254e+02,5.365623e+00 L8.153991e+02,1.179469e+01 L8.323727e+02,7.262949e+00 L8.493464e+02,8.006054e+00 L8.663200e+02,5.200000e+00 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.491734e+02 L3.461121e+01,2.545125e+02 L5.158486e+01,2.533895e+02 L6.855851e+01,2.465955e+02 L8.553216e+01,2.114666e+02 L1.025058e+02,1.954333e+02 L1.194795e+02,1.762972e+02 L1.364531e+02,1.686350e+02 L1.534268e+02,1.646586e+02 L1.704004e+02,1.584968e+02 L1.873741e+02,1.466183e+02 L2.043477e+02,1.489424e+02 L2.213214e+02,1.455937e+02 L2.382950e+02,1.379558e+02 L2.552686e+02,1.273279e+02 L2.722423e+02,1.322356e+02 L2.892159e+02,1.242750e+02 L3.061896e+02,1.205840e+02 L3.231632e+02,1.175008e+02 L3.401369e+02,1.117229e+02 L3.571105e+02,1.054531e+02 L3.740842e+02,1.054713e+02 L3.910578e+02,1.017499e+02 L4.080315e+02,1.066890e+02 L4.250051e+02,9.981562e+01 L4.419788e+02,9.624753e+01 L4.589524e+02,8.897542e+01 L4.759261e+02,9.010807e+01 L4.928997e+02,8.950521e+01 L5.098734e+02,8.814677e+01 L5.268470e+02,8.411271e+01 L5.438207e+02,8.253470e+01 L5.607943e+02,7.624499e+01 L5.777680e+02,7.418234e+01 L5.947416e+02,7.071107e+01 L6.117153e+02,7.422574e+01 L6.286889e+02,6.936834e+01 L6.456626e+02,6.970940e+01 L6.626362e+02,6.285100e+01 L6.796099e+02,6.632482e+01 L6.965835e+02,6.207170e+01

	Mean Loss (#loss): 0.150
	Mean Accuracy (#acc): 94.52%
Results on test-eval [Parallelized]:
	Mean Loss (#loss): 0.377
	Mean Accuracy (#acc): 84.56%


### CNN Model

Below we redefine our CNN model function, with 2 convolution layers, and train on it.

Notice how well it can overfit to the training data ... but it doesn't help the test results. To improve this one needs some careful regularization.

In [8]:
// Conv1DGraph stacks two 1D convolution layers on top of the sequence embeddings (shaped `[batch_size, sequence_len, embed_dim]`).
func Conv1DGraph(ctx *context.Context, embed, mask *Node) *Node {
    g := embed.Graph()
    // 1D Convolution:
    {
        ctx := ctx.In("conv1")
        embed = Activation(embed)
        embed = layers.Dropout(ctx, embed, ConstAsDType(g, DType, *flagDropoutRate))
        embed = layers.Convolution(ctx, embed).KernelSize(7).Filters(*flagTokenEmbeddingSize).Strides(3).Done()
        AssertNoError(g.Error())
        embed = Normalize(ctx, embed)
    }
    {
        ctx := ctx.In("conv2")
        embed = Activation(embed)
        embed = layers.Convolution(ctx, embed).KernelSize(7).Filters(*flagTokenEmbeddingSize).Strides(3).Done()
        AssertNoError(g.Error())
        embed = Normalize(ctx, embed)
    }
    return embed
}

%% --model=cnn --steps=20000 --optimizer=adamw
trainModel()

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Platform: CUDA


Training (20000 steps):  100% [========================================] (430 steps/s) [loss=0.158] [~loss=0.093] [~acc=96.88%]        


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,4.505511e+01 L3.461121e+01,4.500412e+01 L5.158486e+01,4.502111e+01 L6.855851e+01,4.629580e+01 L8.553216e+01,5.854976e+01 L1.025058e+02,5.812485e+01 L1.194795e+02,7.758503e+01 L1.364531e+02,1.099620e+02 L1.534268e+02,1.316146e+02 L1.704004e+02,1.480835e+02 L1.873741e+02,1.598616e+02 L2.043477e+02,1.673057e+02 L2.213214e+02,1.703650e+02 L2.382950e+02,1.736112e+02 L2.552686e+02,1.755487e+02 L2.722423e+02,1.789478e+02 L2.892159e+02,1.810213e+02 L3.061896e+02,1.791178e+02 L3.231632e+02,1.821091e+02 L3.401369e+02,1.784550e+02 L3.571105e+02,1.803075e+02 L3.740842e+02,1.840636e+02 L3.910578e+02,1.763645e+02 L4.080315e+02,1.871908e+02 L4.250051e+02,1.826189e+02 L4.419788e+02,1.865449e+02 L4.589524e+02,1.865449e+02 L4.759261e+02,1.836727e+02 L4.928997e+02,1.796447e+02 L5.098734e+02,1.794917e+02 L5.268470e+02,1.846414e+02 L5.438207e+02,1.877856e+02 L5.607943e+02,1.828909e+02 L5.777680e+02,1.878536e+02 L5.947416e+02,1.874457e+02 L6.117153e+02,1.858991e+02 L6.286889e+02,1.860521e+02 L6.456626e+02,1.862560e+02 L6.626362e+02,1.875307e+02 L6.796099e+02,1.881256e+02 L6.965835e+02,1.866469e+02 L7.135572e+02,1.869699e+02 L7.305308e+02,1.818711e+02 L7.475045e+02,1.856782e+02 L7.644781e+02,1.797296e+02 L7.814518e+02,1.851343e+02 L7.984254e+02,1.832138e+02 L8.153991e+02,1.871568e+02 L8.323727e+02,1.833837e+02 L8.493464e+02,1.835707e+02 L8.663200e+02,1.860861e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,4.500412e+01 L3.461121e+01,4.503809e+01 L5.158486e+01,4.500412e+01 L6.855851e+01,4.758747e+01 L8.553216e+01,6.959701e+01 L1.025058e+02,7.447479e+01 L1.194795e+02,1.063759e+02 L1.364531e+02,1.355236e+02 L1.534268e+02,1.638216e+02 L1.704004e+02,1.741890e+02 L1.873741e+02,1.951618e+02 L2.043477e+02,2.035577e+02 L2.213214e+02,2.092003e+02 L2.382950e+02,2.146390e+02 L2.552686e+02,2.180551e+02 L2.722423e+02,2.222361e+02 L2.892159e+02,2.260092e+02 L3.061896e+02,2.269609e+02 L3.231632e+02,2.310399e+02 L3.401369e+02,2.287965e+02 L3.571105e+02,2.320426e+02 L3.740842e+02,2.357817e+02 L3.910578e+02,2.300372e+02 L4.080315e+02,2.395888e+02 L4.250051e+02,2.382801e+02 L4.419788e+02,2.419002e+02 L4.589524e+02,2.430729e+02 L4.759261e+02,2.419512e+02 L4.928997e+02,2.349149e+02 L5.098734e+02,2.407105e+02 L5.268470e+02,2.451634e+02 L5.438207e+02,2.476448e+02 L5.607943e+02,2.457072e+02 L5.777680e+02,2.485286e+02 L5.947416e+02,2.496333e+02 L6.117153e+02,2.498032e+02 L6.286889e+02,2.501601e+02 L6.456626e+02,2.506700e+02 L6.626362e+02,2.515708e+02 L6.796099e+02,2.523696e+02 L6.965835e+02,2.526075e+02 L7.135572e+02,2.530494e+02 L7.305308e+02,2.514008e+02 L7.475045e+02,2.534233e+02 L7.644781e+02,2.515198e+02 L7.814518e+02,2.540862e+02 L7.984254e+02,2.538312e+02 L8.153991e+02,2.546300e+02 L8.323727e+02,2.543411e+02 L8.493464e+02,2.545960e+02 L8.663200e+02,2.548000e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,5.200000e+00 L3.461121e+01,4.991525e+01 L5.158486e+01,4.738116e+01 L6.855851e+01,4.454072e+01 L8.553216e+01,4.922463e+01 L1.025058e+02,5.089266e+01 L1.194795e+02,6.452660e+01 L1.364531e+02,9.806424e+01 L1.534268e+02,1.329468e+02 L1.704004e+02,1.603937e+02 L1.873741e+02,1.736627e+02 L2.043477e+02,1.825605e+02 L2.213214e+02,1.899518e+02 L2.382950e+02,1.930174e+02 L2.552686e+02,1.958561e+02 L2.722423e+02,1.985437e+02 L2.892159e+02,2.111434e+02 L3.061896e+02,2.076130e+02 L3.231632e+02,2.119558e+02 L3.401369e+02,2.144422e+02 L3.571105e+02,2.177250e+02 L3.740842e+02,2.181715e+02 L3.910578e+02,2.208817e+02 L4.080315e+02,2.227800e+02 L4.250051e+02,2.235637e+02 L4.419788e+02,2.247315e+02 L4.589524e+02,2.259453e+02 L4.759261e+02,2.310325e+02 L4.928997e+02,2.253001e+02 L5.098734e+02,2.289568e+02 L5.268470e+02,2.328535e+02 L5.438207e+02,2.316419e+02 L5.607943e+02,2.347840e+02 L5.777680e+02,2.318668e+02 L5.947416e+02,2.330331e+02 L6.117153e+02,2.337795e+02 L6.286889e+02,2.389528e+02 L6.456626e+02,2.359945e+02 L6.626362e+02,2.369745e+02 L6.796099e+02,2.378437e+02 L6.965835e+02,2.391245e+02


Results on train-eval [Parallelized]:


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.398636e+02 L3.461121e+01,2.395259e+02 L5.158486e+01,2.347734e+02 L6.855851e+01,2.344833e+02 L8.553216e+01,2.342684e+02 L1.025058e+02,2.340470e+02 L1.194795e+02,2.320613e+02 L1.364531e+02,2.112140e+02 L1.534268e+02,1.948614e+02 L1.704004e+02,1.818587e+02 L1.873741e+02,1.699345e+02 L2.043477e+02,1.606397e+02 L2.213214e+02,1.584812e+02 L2.382950e+02,1.544476e+02 L2.552686e+02,1.531060e+02 L2.722423e+02,1.455820e+02 L2.892159e+02,1.486444e+02 L3.061896e+02,1.537765e+02 L3.231632e+02,1.503401e+02 L3.401369e+02,1.624070e+02 L3.571105e+02,1.601683e+02 L3.740842e+02,1.519292e+02 L3.910578e+02,1.750724e+02 L4.080315e+02,1.483346e+02 L4.250051e+02,1.580635e+02 L4.419788e+02,1.515566e+02 L4.589524e+02,1.561011e+02 L4.759261e+02,1.664676e+02 L4.928997e+02,1.703661e+02 L5.098734e+02,1.821730e+02 L5.268470e+02,1.689895e+02 L5.438207e+02,1.598698e+02 L5.607943e+02,1.784582e+02 L5.777680e+02,1.652228e+02 L5.947416e+02,1.598191e+02 L6.117153e+02,1.722631e+02 L6.286889e+02,1.771708e+02 L6.456626e+02,1.739733e+02 L6.626362e+02,1.693501e+02 L6.796099e+02,1.741469e+02 L6.965835e+02,1.728408e+02 L7.135572e+02,1.734143e+02 L7.305308e+02,1.993738e+02 L7.475045e+02,1.797178e+02 L7.644781e+02,2.211414e+02 L7.814518e+02,1.967045e+02 L7.984254e+02,2.067284e+02 L8.153991e+02,1.920824e+02 L8.323727e+02,2.088062e+02 L8.493464e+02,2.100442e+02 L8.663200e+02,1.953941e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.400140e+02 L3.461121e+01,2.393565e+02 L5.158486e+01,2.346363e+02 L6.855851e+01,2.343103e+02 L8.553216e+01,2.338840e+02 L1.025058e+02,2.334131e+02 L1.194795e+02,2.295718e+02 L1.364531e+02,1.946793e+02 L1.534268e+02,1.618868e+02 L1.704004e+02,1.467313e+02 L1.873741e+02,1.184209e+02 L2.043477e+02,1.060912e+02 L2.213214e+02,9.520281e+01 L2.382950e+02,8.656636e+01 L2.552686e+02,8.031523e+01 L2.722423e+02,7.403385e+01 L2.892159e+02,6.710905e+01 L3.061896e+02,6.469449e+01 L3.231632e+02,5.773223e+01 L3.401369e+02,5.970144e+01 L3.571105e+02,5.456558e+01 L3.740842e+02,4.767674e+01 L3.910578e+02,5.710199e+01 L4.080315e+02,4.053902e+01 L4.250051e+02,4.231262e+01 L4.419788e+02,3.529819e+01 L4.589524e+02,3.296709e+01 L4.759261e+02,3.400515e+01 L4.928997e+02,4.547432e+01 L5.098734e+02,3.650455e+01 L5.268470e+02,2.723137e+01 L5.438207e+02,2.271173e+01 L5.607943e+02,2.518675e+01 L5.777680e+02,1.999265e+01 L5.947416e+02,1.824233e+01 L6.117153e+02,1.734975e+01 L6.286889e+02,1.616084e+01 L6.456626e+02,1.510014e+01 L6.626362e+02,1.313717e+01 L6.796099e+02,1.148676e+01 L6.965835e+02,1.098009e+01 L7.135572e+02,1.033914e+01 L7.305308e+02,1.327897e+01 L7.475045e+02,9.664015e+00 L7.644781e+02,1.214961e+01 L7.814518e+02,7.420032e+00 L7.984254e+02,7.585543e+00 L8.153991e+02,5.655403e+00 L8.323727e+02,5.956744e+00 L8.493464e+02,5.290376e+00 L8.663200e+02,5.200000e+00 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.548000e+02 L3.461121e+01,2.349535e+02 L5.158486e+01,2.355006e+02 L6.855851e+01,2.351799e+02 L8.553216e+01,2.346035e+02 L1.025058e+02,2.345253e+02 L1.194795e+02,2.322820e+02 L1.364531e+02,2.187854e+02 L1.534268e+02,1.944421e+02 L1.704004e+02,1.674363e+02 L1.873741e+02,1.491192e+02 L2.043477e+02,1.386437e+02 L2.213214e+02,1.259838e+02 L2.382950e+02,1.210887e+02 L2.552686e+02,1.168667e+02 L2.722423e+02,1.091820e+02 L2.892159e+02,9.353220e+01 L3.061896e+02,9.671721e+01 L3.231632e+02,8.902827e+01 L3.401369e+02,8.778151e+01 L3.571105e+02,8.093332e+01 L3.740842e+02,7.842821e+01 L3.910578e+02,7.462785e+01 L4.080315e+02,7.279030e+01 L4.250051e+02,7.004676e+01 L4.419788e+02,6.660369e+01 L4.589524e+02,6.566080e+01 L4.759261e+02,5.669504e+01 L4.928997e+02,6.526197e+01 L5.098734e+02,5.811495e+01 L5.268470e+02,4.993285e+01 L5.438207e+02,5.272051e+01 L5.607943e+02,4.937814e+01 L5.777680e+02,5.141263e+01 L5.947416e+02,5.030329e+01 L6.117153e+02,4.793418e+01 L6.286889e+02,4.025948e+01 L6.456626e+02,4.720784e+01 L6.626362e+02,4.329128e+01 L6.796099e+02,3.937139e+01 L6.965835e+02,3.800425e+01

	Mean Loss (#loss): 0.028
	Mean Accuracy (#acc): 99.38%
Results on test-eval [Parallelized]:
	Mean Loss (#loss): 0.580
	Mean Accuracy (#acc): 83.21%


### Transformer Model

Finally a Transformer version of the model. 

Notice it's not significantly better than our previous simple Bag-Of-Words model. Likely because there is not enough data for the transformer to make any difference. The success of transformers in large-language-models is in large part due to the training with huge amounts of unsupervised (or self-supervised) data.


In [9]:
var (
    flagMaxAttLen           = flag.Int("max_att_len", 200, "Maximum attention length: input will be split in ranges of this size.")
    flagNumAttHeads         = flag.Int("att_heads", 2, "Number of attention heads, if --model=transformer.")
    flagNumAttLayers        = flag.Int("att_layers", 1, "Number of stacked attention layers, if --model=transformer.")
    flagAttKeyQueryEmbedDim = flag.Int("att_key_dim", 8, "Dimension of the Key/Query attention embedding.")    
)

// TransformerGraph is the part of the model that takes the word/token embeddings to a tranformed
// embedding through attention ready to be pooled and read out.
func TransformerGraph(ctx *context.Context, input, embed, mask *Node) *Node {
    var newEmbed *Node
    if *flagMaxAttLen >= *flagMaxLen {
        // One transformer window covers the whole length, which makes it trivial.
        newEmbed = TransformerLayers(ctx.In("transformer"), embed, mask)
        embed = Add(embed, newEmbed)
        return embed
    }

    // Split embedding in multiple split embeddings and apply transformer in each of them.
    attLen := *flagMaxAttLen
    sequenceFrom := 0
    for {
        // x.shape = [batchSize, sequence, embedding]
        sequenceTo := sequenceFrom + attLen
        if sequenceTo > *flagMaxLen {
            sequenceTo = *flagMaxLen
            sequenceFrom = sequenceTo - attLen
        }
        // part = x[:, sequenceFrom:sequenceTo, :]
        residual := Slice(embed, AxisRange(), AxisRange(sequenceFrom, sequenceTo), AxisRange())
        partMask := Slice(mask, AxisRange(), AxisRange(sequenceFrom, sequenceTo))
        // Reuse "transformer" scope.
        part := TransformerLayers(ctx.In("transformer").Checked(false), residual, partMask)
        part = Add(residual, part)
        if newEmbed == nil {
            newEmbed = part
        } else {
            newEmbed = Add(newEmbed, part)
        }

        if sequenceTo == *flagMaxLen {
            // Reached end of parts.
            break
        }
        sequenceFrom += attLen - 20 // Attention window overlap 10 tokens among themselves.
    }
    embed = newEmbed // Notice shape changed to `[batchSize, maxAttLen, embedDim]`
    return embed
}

// TransformerLayers builds the stacked transformer layers for the model.
func TransformerLayers(ctx *context.Context, embed, mask *Node) *Node {
    g := embed.Graph()
    shape := embed.Shape()
    embedDim := shape.Dimensions[2]

    var dropoutRate *Node
    if *flagDropoutRate > 0 {
        dropoutRate = ConstAsDType(g, DType, *flagDropoutRate)
    }

    // Create positional embedding variable: it is 1 in every axis, but for the
    // sequence dimension -- there will be one embedding per position.
    // Shape: [1, maxLen, embedDim]
    posEmbedShape := shape.Copy()
    posEmbedShape.Dimensions[0] = 1
    posEmbedVar := ctx.VariableWithShape("positional", posEmbedShape)
    posEmbed := posEmbedVar.ValueGraph(g)
    embed = Add(embed, posEmbed) // Just add the embeddings, seems to work well.

    // Add the requested number of attention layers.
    for ii := 0; ii < *flagNumAttLayers; ii++ {
        // Each layer in its own scope.
        ctx := ctx.In(fmt.Sprintf("AttLayer_%d", ii))
        residual := embed
        embed = layers.MultiHeadAttention(ctx, embed, embed, embed, *flagNumAttHeads, *flagAttKeyQueryEmbedDim).
            SetKeyMask(mask).SetQueryMask(mask).
            SetOutputDim(embedDim).
            SetValueHeadDim(embedDim).Done()
        if *flagDropoutRate > 0 {
            embed = layers.Dropout(ctx.In("dropout_1"), embed, dropoutRate)
        }
        embed = Normalize(ctx.In("normalization_1"), embed)
        attentionOutput := embed

        // Transformers recipe: 2 dense layers after attention.
        embed = layers.Dense(ctx.In("ffn_1"), embed, true, embedDim)
        embed = Tanh(embed)
        embed = layers.Dense(ctx.In("ffn_2"), embed, true, embedDim)
        if *flagDropoutRate > 0 {
            embed = layers.Dropout(ctx.In("dropout_1"), embed, dropoutRate)
        }
        embed = Add(embed, attentionOutput)
        embed = Normalize(ctx.In("normalization_2"), embed)

        // Residual connection: not part of the usual transfomer layer ...
        if ii > 0 {
            embed = Add(residual, embed)
        }
    }
    return embed
}

%% --model=transformer --steps=3000 --batch=100 --optimizer=adamw --max_len=200 --max_att_len=200 --att_heads=4 --att_layers=3 --att_key_dim=32 --max_vocab=10000
trainModel()

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Platform: CUDA


Training (3000 steps):  100% [========================================] (7 steps/s) [loss=0.100] [~loss=0.156] [~acc=95.01%]          


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,5.007436e+01 L3.437549e+01,4.597839e+01 L5.135394e+01,4.597839e+01 L6.833240e+01,4.597839e+01 L8.531086e+01,5.505837e+01 L1.022893e+02,4.420226e+01 L1.192678e+02,1.156097e+02 L1.362462e+02,1.585449e+02 L1.532247e+02,1.761611e+02 L1.702032e+02,1.858211e+02 L1.871816e+02,1.888840e+02 L2.041601e+02,1.937411e+02 L2.211385e+02,1.955716e+02 L2.381170e+02,1.988158e+02 L2.550955e+02,1.990333e+02 L2.720739e+02,1.783541e+02 L2.890524e+02,1.973659e+02 L3.060308e+02,1.982540e+02 L3.230093e+02,2.012625e+02 L3.399878e+02,1.962966e+02 L3.569662e+02,2.040354e+02 L3.739447e+02,1.948830e+02 L3.909231e+02,2.033830e+02 L4.079016e+02,1.989970e+02 L4.248801e+02,1.975290e+02 L4.418585e+02,2.049416e+02 L4.588370e+02,2.062647e+02 L4.758154e+02,2.076964e+02 L4.927939e+02,2.035461e+02 L5.097724e+02,2.044704e+02 L5.267508e+02,2.091282e+02 L5.437293e+02,2.083670e+02 L5.607077e+02,2.060290e+02 L5.776862e+02,2.020781e+02 L5.946646e+02,2.038361e+02 L6.116431e+02,2.057391e+02 L6.286216e+02,2.034736e+02 L6.456000e+02,2.075152e+02 L6.625785e+02,2.057028e+02 L6.795569e+02,2.051772e+02 L6.965354e+02,2.061921e+02 L7.135139e+02,2.026580e+02 L7.304923e+02,2.050685e+02 L7.474708e+02,2.043979e+02 L7.644492e+02,2.057028e+02 L7.814277e+02,2.063371e+02 L7.984062e+02,2.058297e+02 L8.153846e+02,2.048691e+02 L8.323631e+02,2.071708e+02 L8.493415e+02,2.035461e+02 L8.663200e+02,1.977646e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,5.059993e+01 L3.437549e+01,4.597839e+01 L5.135394e+01,4.597839e+01 L6.833240e+01,4.597839e+01 L8.531086e+01,5.360848e+01 L1.022893e+02,6.683881e+01 L1.192678e+02,1.277345e+02 L1.362462e+02,1.736057e+02 L1.532247e+02,1.976559e+02 L1.702032e+02,2.036911e+02 L1.871816e+02,2.107593e+02 L2.041601e+02,2.159608e+02 L2.211385e+02,2.163777e+02 L2.381170e+02,2.224129e+02 L2.550955e+02,2.246421e+02 L2.720739e+02,2.068446e+02 L2.890524e+02,2.252765e+02 L3.060308e+02,2.272519e+02 L3.230093e+02,2.302605e+02 L3.399878e+02,2.279044e+02 L3.569662e+02,2.335409e+02 L3.739447e+02,2.284300e+02 L3.909231e+02,2.355707e+02 L4.079016e+02,2.332327e+02 L4.248801e+02,2.342296e+02 L4.418585e+02,2.387605e+02 L4.588370e+02,2.403735e+02 L4.758154e+02,2.418959e+02 L4.927939e+02,2.411709e+02 L5.097724e+02,2.420590e+02 L5.267508e+02,2.446688e+02 L5.437293e+02,2.453213e+02 L5.607077e+02,2.461369e+02 L5.776862e+02,2.432189e+02 L5.946646e+02,2.427658e+02 L6.116431e+02,2.455750e+02 L6.286216e+02,2.453213e+02 L6.456000e+02,2.494173e+02 L6.625785e+02,2.469343e+02 L6.795569e+02,2.481123e+02 L6.965354e+02,2.500516e+02 L7.135139e+02,2.472243e+02 L7.304923e+02,2.493991e+02 L7.474708e+02,2.481123e+02 L7.644492e+02,2.521721e+02 L7.814277e+02,2.543107e+02 L7.984062e+02,2.532776e+02 L8.153846e+02,2.518277e+02 L8.323631e+02,2.548000e+02 L8.493415e+02,2.438895e+02 L8.663200e+02,2.416422e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,5.200000e+00 L3.437549e+01,6.500834e+01 L5.135394e+01,5.425255e+01 L6.833240e+01,5.575020e+01 L8.531086e+01,5.701864e+01 L1.022893e+02,5.902854e+01 L1.192678e+02,7.200031e+01 L1.362462e+02,1.080677e+02 L1.532247e+02,1.403431e+02 L1.702032e+02,1.631510e+02 L1.871816e+02,1.832972e+02 L2.041601e+02,1.957751e+02 L2.211385e+02,2.011245e+02 L2.381170e+02,2.061243e+02 L2.550955e+02,2.105062e+02 L2.720739e+02,2.123726e+02 L2.890524e+02,2.150753e+02 L3.060308e+02,2.190480e+02 L3.230093e+02,2.228384e+02 L3.399878e+02,2.242765e+02 L3.569662e+02,2.259290e+02 L3.739447e+02,2.263707e+02 L3.909231e+02,2.295381e+02 L4.079016e+02,2.311053e+02 L4.248801e+02,2.319233e+02 L4.418585e+02,2.304701e+02 L4.588370e+02,2.340389e+02 L4.758154e+02,2.348556e+02 L4.927939e+02,2.355303e+02 L5.097724e+02,2.374146e+02 L5.267508e+02,2.391440e+02 L5.437293e+02,2.402761e+02 L5.607077e+02,2.385771e+02 L5.776862e+02,2.396321e+02 L5.946646e+02,2.416140e+02 L6.116431e+02,2.417526e+02 L6.286216e+02,2.421201e+02 L6.456000e+02,2.430999e+02 L6.625785e+02,2.446716e+02 L6.795569e+02,2.451958e+02 L6.965354e+02,2.466139e+02

<path d="M1.768000e+01,2.419523e+02 L3.437549e+01,2.422503e+02 L5.135394e+01,2.430361e+02 L6.833240e+01,2.426466e+02 L8.531086e+01,2.422361e+02 L1.022893e+02,2.431965e+02 L1.192678e+02,2.229813e+02 L1.362462e+02,1.748712e+02 L1.532247e+02,1.559296e+02 L1.702032e+02,1.376258e+02 L1.871816e+02,1.383891e+02 L2.041601e+02,1.211722e+02 L2.211385e+02,1.162501e+02 L2.381170e+02,1.121135e+02 L2.550955e+02,1.082906e+02 L2.720739e+02,1.369469e+02 L2.890524e+02,1.095590e+02 L3.060308e+02,1.107309e+02 L3.230093e+02,1.054677e+02 L3.399878e+02,1.142984e+02 L3.569662e+02,9.848454e+01 L3.739447e+02,1.107189e+02 L3.909231e+02,9.832093e+01 L4.079016e+02,1.030026e+02 L4.248801e+02,1.040363e+02 L4.418585e+02,9.298591e+01 L4.588370e+02,9.943880e+01 L4.758154e+02,9.514068e+01 L4.927939e+02,9.582459e+01 L5.097724e+02,1.006575e+02 L5.267508e+02,9.877368e+01 L5.437293e+02,9.821070e+01 L5.607077e+02,9.744154e+01 L5.776862e+02,1.051939e+02 L5.946646e+02,1.022924e+02 L6.116431e+02,1.013172e+02 L6.286216e+02,1.097179e+02 L6.456000e+02,1.032743e+02 L6.625785e+02,1.191537e+02 L6.795569e+02,1.084348e+02 L6.965354e+02,1.148004e+02 L7.135139e+02,1.123337e+02 L7.304923e+02,1.150215e+02 L7.474708e+02,1.209057e+02 L7.644492e+02,1.144077e+02 L7.814277e+02,1.136645e+02 L7.984062e+02,1.108075e+02 L8.153846e+02,1.249443e+02 L8.323631e+02,1.221108e+02 L8.493415e+02,1.350320e+02 L8.663200e+02,1.448155e+02 " vector-effect="non-scaling-stroke"/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.421320e+02 L3.437549e+01,2.421897e+02 L5.135394e+01,2.427960e+02 L6.833240e+01,2.414564e+02 L8.531086e+01,2.407119e+02 L1.022893e+02,2.407862e+02 L1.192678e+02,2.182518e+02 L1.362462e+02,1.581251e+02 L1.532247e+02,1.266425e+02 L1.702032e+02,1.115424e+02 L1.871816e+02,1.012854e+02 L2.041601e+02,8.745112e+01 L2.211385e+02,8.980373e+01 L2.381170e+02,7.593345e+01 L2.550955e+02,7.058481e+01 L2.720739e+02,8.871194e+01 L2.890524e+02,6.398125e+01 L3.060308e+02,6.051080e+01 L3.230093e+02,5.538951e+01 L3.399878e+02,5.700097e+01 L3.569662e+02,5.005526e+01 L3.739447e+02,5.362352e+01 L3.909231e+02,4.534999e+01 L4.079016e+02,4.651801e+01 L4.248801e+02,4.510779e+01 L4.418585e+02,3.947913e+01 L4.588370e+02,3.469810e+01 L4.758154e+02,3.292294e+01 L4.927939e+02,3.520156e+01 L5.097724e+02,3.077351e+01 L5.267508e+02,2.703743e+01 L5.437293e+02,2.755898e+01 L5.607077e+02,2.619488e+01 L5.776862e+02,2.724448e+01 L5.946646e+02,2.703480e+01 L6.116431e+02,2.308080e+01 L6.286216e+02,2.231119e+01 L6.456000e+02,1.949713e+01 L6.625785e+02,1.891537e+01 L6.795569e+02,1.714317e+01 L6.965354e+02,1.469923e+01 L7.135139e+02,1.792955e+01 L7.304923e+02,1.370940e+01 L7.474708e+02,1.428216e+01 L7.644492e+02,1.094021e+01 L7.814277e+02,8.214343e+00 L7.984062e+02,1.042383e+01 L8.153846e+02,8.933833e+00 L8.323631e+02,5.200000e+00 L8.493415e+02,2.472776e+01 L8.663200e+02,2.259024e+01 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.548000e+02 L3.437549e+01,2.411600e+02 L5.135394e+01,2.422309e+02 L6.833240e+01,2.417564e+02 L8.531086e+01,2.413374e+02 L1.022893e+02,2.410639e+02 L1.192678e+02,2.376869e+02 L1.362462e+02,2.161804e+02 L1.532247e+02,1.875996e+02 L1.702032e+02,1.641345e+02 L1.871816e+02,1.383605e+02 L2.041601e+02,1.215505e+02 L2.211385e+02,1.132092e+02 L2.381170e+02,1.053283e+02 L2.550955e+02,9.680983e+01 L2.720739e+02,9.167487e+01 L2.890524e+02,8.704630e+01 L3.060308e+02,7.883634e+01 L3.230093e+02,7.200112e+01 L3.399878e+02,6.827655e+01 L3.569662e+02,6.644677e+01 L3.739447e+02,6.501678e+01 L3.909231e+02,5.899399e+01 L4.079016e+02,5.455620e+01 L4.248801e+02,5.237708e+01 L4.418585e+02,5.396906e+01 L4.588370e+02,4.824574e+01 L4.758154e+02,4.706173e+01 L4.927939e+02,4.593958e+01 L5.097724e+02,4.135253e+01 L5.267508e+02,3.770884e+01 L5.437293e+02,3.527813e+01 L5.607077e+02,3.659473e+01 L5.776862e+02,3.676736e+01 L5.946646e+02,3.334754e+01 L6.116431e+02,3.230271e+01 L6.286216e+02,3.056621e+01 L6.456000e+02,2.989693e+01 L6.625785e+02,2.625337e+01 L6.795569e+02,2.494737e+01 L6.965354e+02,2.335853e+01


Results on train-eval [Parallelized]:
	Mean Loss (#loss): 0.177
	Mean Accuracy (#acc): 93.18%
Results on test-eval [Parallelized]:
	Mean Loss (#loss): 0.464
	Mean Accuracy (#acc): 83.50%
